# Lightweight Fine-Tuning Project

Our goal is to compare traditional Full fine-tuning BERT model on sentiment analysis task (actually DistilBERT) with PEFT Bert model on the same tasks using the Hugging Face Transformers library. For evaluation we use accurary approach and We will use the IMDB movie review dataset to train and evaluate Bert model. The IMDB dataset contains movie reviews that are labeled as either positive or negative.

* PEFT technique : LoRA
* Model : BERT 
* Evaluation approach : Accuracy
* Fine-tuning dataset :  IMDB dataset

In [ ]:
# Install and update some packages first

### then restart the kernel to use updated packages ###

# !pip install -q --upgrade datasets transformers[torch] peft 
%pip install peft

## Loading and Evaluating a Foundation Model

In the cells below, We load Bert pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [3]:
# Import the datasets packages from HugginFace
from datasets import load_dataset_builder, load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          DataCollatorWithPadding, Trainer, TrainingArguments)
import os
import torch
import numpy as np
import pandas as pd

print("current directory is :", os.getcwd())

# Attempt GPU; if not, stay on CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

current directory is : d:\M2\deep-learing\transformers
cuda:0


In [4]:
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]

dataset={split:ds for split, ds in zip(splits, load_dataset("imdb",split=splits))}

In [5]:
# check data
dataset

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 })}

In [6]:
# We choose smaller subset of dataset to fine-tune to reduce the time it takes to train
ds={split:dataset[split].shuffle(seed=42).select(range(500)) for split in splits}
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [7]:
# show few lines of text
ds['train']["text"][:3]

['There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...',
 'This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds m

In [8]:
label={0: "NEGATIVE", 1: "POSITIVE"}
# show few lines of label (1=pos,0=neg)
ds['train']["label"][:3]

[1, 1, 0]

In [9]:
# show few lines of text
ds['test']["text"][:3]

["<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness!<br /><br />I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there.<br /><br />Sinc

In [10]:
# show few lines of label (1=pos,0=neg)
ds['test']["label"][:3]

[1, 1, 0]

###  Tokenize text

Models cannot process raw text, so you’ll need to convert the text into numbers (integers). Tokenization provides a way to do this by dividing text into individual words called tokens. Tokens are finally converted to numbers.

In [11]:
# Load the tokenizer for Bert
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
def tokenization(example):
    """
    tokenize your entire dataset

    """
    return tokenizer(example["text"], padding=True, truncation=True)

In [13]:
tokenize_ds={split:ds[split].map(tokenization, batched=True) for split in splits}

Map: 100%|██████████| 500/500 [00:00<00:00, 2676.53 examples/s]


In [14]:
# check
tokenize_ds

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 500
 })}

### Train Bert model

Now it's time to train our model. We'll use the Trainer class from the 🤗 Transformers library to do this. The Trainer class provides a high-level API that abstracts away a lot of the training loop.

First we'll define a function to compute our accuracy metreic then we make the Trainer.

Let's take this opportunity to learn about the DataCollator. According to the HuggingFace documentation:

Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.

To be able to build batches, data collators may apply some processing (like padding).

In [15]:
bert_model=AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=2,
        id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
        label2id={"NEGATIVE": 0, "POSITIVE": 1},
       
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args=TrainingArguments(
    output_dir="./result_non_PEFT/sentiment_analysis",
    learning_rate=1e-4,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    num_train_epochs=3,
    weight_decay=0.001,  
    logging_strategy="epoch",
    eval_strategy="epoch",         
    save_strategy="epoch",         
    load_best_model_at_end=True
)

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [20]:
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=tokenize_ds["train"],
    eval_dataset=tokenize_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, return_tensors="pt"),
    compute_metrics=compute_metrics
    )

In [21]:
# run trainer to train bert model
trainer.train()

 33%|███▎      | 32/96 [00:20<00:32,  1.94it/s]

{'loss': 0.5799, 'grad_norm': 0.9860185384750366, 'learning_rate': 6.666666666666667e-05, 'epoch': 1.0}


                                               
 33%|███▎      | 32/96 [00:28<00:32,  1.94it/s]

{'eval_loss': 0.48523396253585815, 'eval_accuracy': 0.804, 'eval_runtime': 7.2649, 'eval_samples_per_second': 68.824, 'eval_steps_per_second': 4.405, 'epoch': 1.0}


 67%|██████▋   | 64/96 [00:49<00:16,  1.92it/s]

...

                                               
100%|██████████| 96/96 [01:26<00:00,  1.91it/s]

{'eval_loss': 0.41351214051246643, 'eval_accuracy': 0.882, 'eval_runtime': 7.2747, 'eval_samples_per_second': 68.731, 'eval_steps_per_second': 4.399, 'epoch': 3.0}


100%|██████████| 96/96 [01:28<00:00,  1.09it/s]

{'train_runtime': 88.0407, 'train_samples_per_second': 17.038, 'train_steps_per_second': 1.09, 'train_loss': 0.31628889342149097, 'epoch': 3.0}


TrainOutput(global_step=96, training_loss=0.31628889342149097, metrics={'train_runtime': 88.0407, 'train_samples_per_second': 17.038, 'train_steps_per_second': 1.09, 'total_flos': 198701097984000.0, 'train_loss': 0.31628889342149097, 'epoch': 3.0})

### Evaluate the model

Evaluating the model is as simple as calling the evaluate method on the trainer object. This will run the model on the test set and compute the metrics we specified in the compute_metrics function.

In [22]:
# Evaluate the model
trainer.evaluate()

100%|██████████| 32/32 [00:07<00:00,  4.54it/s]


{'eval_loss': 0.4032055139541626,
 'eval_accuracy': 0.87,
 'eval_runtime': 7.5302,
 'eval_samples_per_second': 66.399,
 'eval_steps_per_second': 4.25,
 'epoch': 3.0}

In [23]:
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(tokenize_ds["test"])
df = df[["text", "label"]]

# Replace <br /> tags in the text with spaces
df["text"] = df["text"].str.replace("<br />", " ")

# display few row of raw data
df.head()

,text,label
0,"When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong? Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness! I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there. Since I didn't understand why the cover said the film was about sisters fighting over land -they weren't fighting each other at all- I watched it a second time. Then I was able to see that if one hadn't lived a similar story, one would easily miss the overwhelming undercurrent of dread and fear and the deep bond between the sisters that runs through it all. That is exactly the reason why people in general often overlook the truth about their neighbors for instance. But yet another reason why this movie is so perfect! I don't give a rat's ass (pardon my French) about to what extend the King Lear story is followed. All I know is that I can honestly say: this movie has changed my life. Keep up the good work guys, you CAN and DO make a difference.",1
1,"This is the latest entry in the long series of films with the French agent, O.S.S. 117 (the French answer to James Bond). The series was launched in the early 1950's, and spawned at least eight films (none of which was ever released in the U.S.). 'O.S.S.117:Cairo,Nest Of Spies' is a breezy little comedy that should not...repeat NOT, be taken too seriously. Our protagonist finds himself in the middle of a spy chase in Egypt (with Morroco doing stand in for Egypt) to find out about a long lost friend. What follows is the standard James Bond/Inspector Cloussou kind of antics. Although our man is something of an overt xenophobe,sexist,homophobe, it's treated as pure farce (as I said, don't take it too seriously). Although there is a bit of rough language & cartoon violence, it's basically okay for older kids (ages 12 & up). As previously stated in the subject line, just sit back,pass the popcorn & just enjoy.",1
2,"This movie was so frustrating. Everything seemed energetic and I was totally prepared to have a good time. I at least thought I'd be able to stand it. But, I was wrong. First, the weird looping? It was like watching ""America's Funniest Home Videos"". The damn parents. I hated them so much. The stereo-typical Latino family? I need to speak with the person responsible for this. We need to have a talk. That little girl who was always hanging on someone? I just hated her and had to mention it. Now, the final scene transcends, I must say. It's so gloriously bad and full of badness that it is a movie of its own. What crappy dancing. Horrible and beautiful at once.",0
3,"I was truly and wonderfully surprised at ""O' Brother, Where Art Thou?"" The video store was out of all the movies I was planning on renting, so then I came across this. I came home and as I watched I became engrossed and found myself laughing out loud. The Coen's have made a magnificiant film again. But I think the first time you watch this movie, you get to know the characters. The second time, now that you know them, you laugh sooo hard it could hurt you. I strongly would reccomend ANYONE seeing this because if you are not, you are truly missing a film gem for the ages. 10/10",1
4,"This movie spends most of its time preaching that it is the script that makes the mov

In [24]:
# Add the model predictions to the dataframe
predictions = trainer.predict(tokenize_ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head()

100%|██████████| 32/32 [00:07<00:00,  4.55it/s]


,text,label,predicted_label
0,"When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong? Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness! I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there. Since I didn't understand why the cover said the film was about sisters fighting over land -they weren't fighting each other at all- I watched it a second time. Then I was able to see that if one hadn't lived a similar story, one would easily miss the overwhelming undercurrent of dread and fear and the deep bond between the sisters that runs through it all. That is exactly the reason why people in general often overlook the truth about their neighbors for instance. But yet another reason why this movie is so perfect! I don't give a rat's ass (pardon my French) about to what extend the King Lear story is followed. All I know is that I can honestly say: this movie has changed my life. Keep up the good work guys, you CAN and DO make a difference.",1,1
1,"This is the latest entry in the long series of films with the French agent, O.S.S. 117 (the French answer to James Bond). The series was launched in the early 1950's, and spawned at least eight films (none of which was ever released in the U.S.). 'O.S.S.117:Cairo,Nest Of Spies' is a breezy little comedy that should not...repeat NOT, be taken too seriously. Our protagonist finds himself in the middle of a spy chase in Egypt (with Morroco doing stand in for Egypt) to find out about a long lost friend. What follows is the standard James Bond/Inspector Cloussou kind of antics. Although our man is something of an overt xenophobe,sexist,homophobe, it's treated as pure farce (as I said, don't take it too seriously). Although there is a bit of rough language & cartoon violence, it's basically okay for older kids (ages 12 & up). As previously stated in the subject line, just sit back,pass the popcorn & just enjoy.",1,0
2,"This movie was so frustrating. Everything seemed energetic and I was totally prepared to have a good time. I at least thought I'd be able to stand it. But, I was wrong. First, the weird looping? It was like watching ""America's Funniest Home Videos"". The damn parents. I hated them so much. The stereo-typical Latino family? I need to speak with the person responsible for this. We need to have a talk. That little girl who was always hanging on someone? I just hated her and had to mention it. Now, the final scene transcends, I must say. It's so gloriously bad and full of badness that it is a movie of its own. What crappy dancing. Horrible and beautiful at once.",0,0
3,"I was truly and wonderfully surprised at ""O' Brother, Where Art Thou?"" The video store was out of all the movies I was planning on renting, so then I came across this. I came home and as I watched I became engrossed and found myself laughing out loud. The Coen's have made a magnificiant film again. But I think the first time you watch this movie, you get to know the characters. The second time, now that you know them, you laugh sooo hard it could hurt you. I strongly would reccomend ANYONE seeing this because if you are not, you are truly missing a film gem for the ages. 10/10",1,0
4,"This movie spends most of its time preaching that it is the s

## Performing Parameter-Efficient Fine-Tuning

In the cells below, We create a PEFT model from the loaded model, run a training loop, and save the PEFT model weights.

In [25]:
from peft import LoraConfig, TaskType, get_peft_model

Lora_config = LoraConfig(task_type=TaskType.SEQ_CLS,
                         target_modules=["q_lin","k_lin","v_lin"],
                         inference_mode=False, 
                         r=8, lora_alpha=32, 
                         lora_dropout=0.1
                        )

In [26]:
# create a PeftModel with the get_peft_model() function It takes a base model, which you can load from the Transformers library 
# and the LoraConfig containing the parameters for how to configure a model for training with LoRA.

peft_bert_model=get_peft_model(bert_model,Lora_config)

peft_bert_model


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=76

In [27]:
peft_bert_model.print_trainable_parameters()

trainable params: 813,314 || all params: 67,768,324 || trainable%: 1.2001


### Train peft model

Each PEFT method is defined by a PeftConfig class that stores all the important parameters for building a PeftModel. For example, to train with LoRA, load and create a LoraConfig class and specify the following parameters:

- task_type: the task to train for (text classification language modeling in this case)
- inference_mode: whether you’re using the model for inference or not
- r: the dimension of the low-rank matrices
- lora_alpha: the scaling factor for the low-rank matrices
- lora_dropout: the dropout probability of the LoRA layers

In [28]:
training_args_peft=TrainingArguments(
    output_dir="./result_PEFT/sentiment_analysis",
    learning_rate=1e-4,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",         
    save_strategy="epoch",         
    load_best_model_at_end=True
)

In [30]:
peft_trainer = Trainer(
    model=peft_bert_model,
    args=training_args_peft,
    train_dataset=tokenize_ds["train"],
    eval_dataset=tokenize_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, return_tensors="pt"),
    compute_metrics=compute_metrics
    )

In [31]:
# run trainer to train bert model
peft_trainer.train()

 10%|█         | 32/320 [00:24<01:55,  2.50it/s]

{'eval_loss': 0.4556330740451813, 'eval_accuracy': 0.852, 'eval_runtime': 7.6717, 'eval_samples_per_second': 65.174, 'eval_steps_per_second': 4.171, 'epoch': 1.0}


 20%|██        | 64/320 [00:48<01:43,  2.47it/s]

{'eval_loss': 0.42972803115844727, 'eval_accuracy': 0.858, 'eval_runtime': 7.7307, 'eval_samples_per_second': 64.677, 'eval_steps_per_second': 4.139, 'epoch': 2.0}


 30%|███       | 96/320 [01:12<01:31,  2.45it/s]

...

100%|██████████| 320/320 [04:06<00:00,  2.42it/s]

{'eval_loss': 0.438707172870636, 'eval_accuracy': 0.864, 'eval_runtime': 7.845, 'eval_samples_per_second': 63.735, 'eval_steps_per_second': 4.079, 'epoch': 10.0}


100%|██████████| 320/320 [04:06<00:00,  1.30it/s]

{'train_runtime': 246.5087, 'train_samples_per_second': 20.283, 'train_steps_per_second': 1.298, 'train_loss': 0.08165293335914611, 'epoch': 10.0}


TrainOutput(global_step=320, training_loss=0.08165293335914611, metrics={'train_runtime': 246.5087, 'train_samples_per_second': 20.283, 'train_steps_per_second': 1.298, 'total_flos': 674829496320000.0, 'train_loss': 0.08165293335914611, 'epoch': 10.0})

In [32]:
# Evaluate the peft model
peft_trainer.evaluate()

100%|██████████| 32/32 [00:07<00:00,  4.40it/s]


{'eval_loss': 0.42245352268218994,
 'eval_accuracy': 0.864,
 'eval_runtime': 8.6616,
 'eval_samples_per_second': 57.726,
 'eval_steps_per_second': 3.694,
 'epoch': 10.0}

In [33]:
pd.set_option("display.max_colwidth", None)

dfs = pd.DataFrame(tokenize_ds["test"])
dfs = dfs[["text", "label"]]

# Replace <br /> tags in the text with spaces
dfs["text"] = dfs["text"].str.replace("<br />", " ")

dfs.head()

,text,label
0,"When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong? Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness! I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there. Since I didn't understand why the cover said the film was about sisters fighting over land -they weren't fighting each other at all- I watched it a second time. Then I was able to see that if one hadn't lived a similar story, one would easily miss the overwhelming undercurrent of dread and fear and the deep bond between the sisters that runs through it all. That is exactly the reason why people in general often overlook the truth about their neighbors for instance. But yet another reason why this movie is so perfect! I don't give a rat's ass (pardon my French) about to what extend the King Lear story is followed. All I know is that I can honestly say: this movie has changed my life. Keep up the good work guys, you CAN and DO make a difference.",1
1,"This is the latest entry in the long series of films with the French agent, O.S.S. 117 (the French answer to James Bond). The series was launched in the early 1950's, and spawned at least eight films (none of which was ever released in the U.S.). 'O.S.S.117:Cairo,Nest Of Spies' is a breezy little comedy that should not...repeat NOT, be taken too seriously. Our protagonist finds himself in the middle of a spy chase in Egypt (with Morroco doing stand in for Egypt) to find out about a long lost friend. What follows is the standard James Bond/Inspector Cloussou kind of antics. Although our man is something of an overt xenophobe,sexist,homophobe, it's treated as pure farce (as I said, don't take it too seriously). Although there is a bit of rough language & cartoon violence, it's basically okay for older kids (ages 12 & up). As previously stated in the subject line, just sit back,pass the popcorn & just enjoy.",1
2,"This movie was so frustrating. Everything seemed energetic and I was totally prepared to have a good time. I at least thought I'd be able to stand it. But, I was wrong. First, the weird looping? It was like watching ""America's Funniest Home Videos"". The damn parents. I hated them so much. The stereo-typical Latino family? I need to speak with the person responsible for this. We need to have a talk. That little girl who was always hanging on someone? I just hated her and had to mention it. Now, the final scene transcends, I must say. It's so gloriously bad and full of badness that it is a movie of its own. What crappy dancing. Horrible and beautiful at once.",0
3,"I was truly and wonderfully surprised at ""O' Brother, Where Art Thou?"" The video store was out of all the movies I was planning on renting, so then I came across this. I came home and as I watched I became engrossed and found myself laughing out loud. The Coen's have made a magnificiant film again. But I think the first time you watch this movie, you get to know the characters. The second time, now that you know them, you laugh sooo hard it could hurt you. I strongly would reccomend ANYONE seeing this because if you are not, you are truly missing a film gem for the ages. 10/10",1
4,"This movie spends most of its time preaching that it is the script that makes the mov

In [34]:
# Add the model predictions to the dataframe
predictions = peft_trainer.predict(tokenize_ds["test"])
dfs["predicted_label"] = np.argmax(predictions[0], axis=1)

dfs.head()

100%|██████████| 32/32 [00:07<00:00,  4.41it/s]


,text,label,predicted_label
0,"When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong? Very quickly, however, I realized that this story was about A Thousand Other Things besides just Acres. I started crying and couldn't stop until long after the movie ended. Thank you Jane, Laura and Jocelyn, for bringing us such a wonderfully subtle and compassionate movie! Thank you cast, for being involved and portraying the characters with such depth and gentleness! I recognized the Angry sister; the Runaway sister and the sister in Denial. I recognized the Abusive Husband and why he was there and then the Father, oh oh the Father... all superbly played. I also recognized myself and this movie was an eye-opener, a relief, a chance to face my OWN truth and finally doing something about it. I truly hope A Thousand Acres has had the same effect on some others out there. Since I didn't understand why the cover said the film was about sisters fighting over land -they weren't fighting each other at all- I watched it a second time. Then I was able to see that if one hadn't lived a similar story, one would easily miss the overwhelming undercurrent of dread and fear and the deep bond between the sisters that runs through it all. That is exactly the reason why people in general often overlook the truth about their neighbors for instance. But yet another reason why this movie is so perfect! I don't give a rat's ass (pardon my French) about to what extend the King Lear story is followed. All I know is that I can honestly say: this movie has changed my life. Keep up the good work guys, you CAN and DO make a difference.",1,1
1,"This is the latest entry in the long series of films with the French agent, O.S.S. 117 (the French answer to James Bond). The series was launched in the early 1950's, and spawned at least eight films (none of which was ever released in the U.S.). 'O.S.S.117:Cairo,Nest Of Spies' is a breezy little comedy that should not...repeat NOT, be taken too seriously. Our protagonist finds himself in the middle of a spy chase in Egypt (with Morroco doing stand in for Egypt) to find out about a long lost friend. What follows is the standard James Bond/Inspector Cloussou kind of antics. Although our man is something of an overt xenophobe,sexist,homophobe, it's treated as pure farce (as I said, don't take it too seriously). Although there is a bit of rough language & cartoon violence, it's basically okay for older kids (ages 12 & up). As previously stated in the subject line, just sit back,pass the popcorn & just enjoy.",1,1
2,"This movie was so frustrating. Everything seemed energetic and I was totally prepared to have a good time. I at least thought I'd be able to stand it. But, I was wrong. First, the weird looping? It was like watching ""America's Funniest Home Videos"". The damn parents. I hated them so much. The stereo-typical Latino family? I need to speak with the person responsible for this. We need to have a talk. That little girl who was always hanging on someone? I just hated her and had to mention it. Now, the final scene transcends, I must say. It's so gloriously bad and full of badness that it is a movie of its own. What crappy dancing. Horrible and beautiful at once.",0,0
3,"I was truly and wonderfully surprised at ""O' Brother, Where Art Thou?"" The video store was out of all the movies I was planning on renting, so then I came across this. I came home and as I watched I became engrossed and found myself laughing out loud. The Coen's have made a magnificiant film again. But I think the first time you watch this movie, you get to know the characters. The second time, now that you know them, you laugh sooo hard it could hurt you. I strongly would reccomend ANYONE seeing this because if you are not, you are truly missing a film gem for the ages. 10/10",1,1
4,"This movie spends most of its time preaching that it is the s

## Performing Inference with a PEFT Model

In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [35]:
# save the PEFT model.
peft_bert_model.save_pretrained("lora_bert_model")

# save bert model
bert_model.save_pretrained("Bert_Model")

In [36]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer
import torch

model = AutoPeftModelForSequenceClassification.from_pretrained("lora_bert_model")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = model.to(device)
model.eval()

input_text = "I love Udacity deep learning course"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

predictions= model(**inputs).logits.cpu().detach().numpy()

print('The sentiment anlysis of this `{input_text}` is:',label[np.argmax(predictions[0])])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The sentiment anlysis of this `{input_text}` is: POSITIVE


In [37]:
BertModel = AutoModelForSequenceClassification.from_pretrained("Bert_Model")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Model = BertModel.to(device)
Model.eval()

input_text = "I love Udacity deep learning course"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

predictions= Model(**inputs).logits.cpu().detach().numpy()

print(f'The sentiment anlysis of this `{input_text}` is:',label[np.argmax(predictions[0])])

Some weights of the model checkpoint at Bert_Model were not used when initializing DistilBertForSequenceClassification: ['classifier.modules_to_save.default.bias', 'classifier.modules_to_save.default.weight', 'classifier.original_module.bias', 'classifier.original_module.weight', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.k_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.k_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.q_lin.base_layer.weight', 'distilbert.transformer.layer.0.attention.q_lin.lora_A.default.weight', 'distilbert.transformer.layer.0.attention.q_lin.lora_B.default.weight', 'distilbert.transformer.layer.0.attention.v_lin.base_layer.bias', 'distilbert.transformer.layer.0.attention.v_lin.base_layer.weight', 'distilbert.transformer.layer

The sentiment anlysis of this `I love Udacity deep learning course` is: POSITIVE
